In [1]:
import pandas

In [2]:
from peanut.smpi_hpl import SMPIHPL
from peanut.peanut import ExpFile, RunError
if True:
    job = SMPIHPL.oarsub_cluster(cluster='dahu', username='tocornebize', walltime='04:00:00', deploy='debian9-x64-base', nb_nodes=1)
    job.setup()
else:
    job = SMPIHPL.get_jobs('nancy', 'tocornebize')[0]
    job.nodes.run('pwd')

[2018-12-20 17:38:38][DEBUG] [frontend | /home/tocornebize] oarsub -n "🥐" -q default -t deploy  -l "{cluster in ('dahu')}/nodes=1,walltime=04:00:00" -r "2018-12-20 17:38:38"
[2018-12-20 17:38:42][DEBUG] [frontend | /home/tocornebize] oarstat -fJ -j 1829997
[2018-12-20 17:38:48][DEBUG] [frontend | /home/tocornebize] oarstat -fJ -j 1829997
[2018-12-20 17:38:59][DEBUG] [frontend | /home/tocornebize] oarstat -fJ -j 1829997
[2018-12-20 17:39:22][DEBUG] [frontend | /home/tocornebize] oarstat -fJ -j 1829997
[2018-12-20 17:40:09][DEBUG] [frontend | /home/tocornebize] oarstat -fJ -j 1829997
[2018-12-20 17:40:09][DEBUG] [frontend | /home/tocornebize] kadeploy3 -k -f /var/lib/oar/1829997 -e debian9-x64-base
[2018-12-20 17:43:33][DEBUG] [allnodes | /tmp] echo "hello world"
[2018-12-20 17:43:33][DEBUG] [allnodes | /tmp] rm -rf /tmp/*
[2018-12-20 17:43:33][DEBUG] [director | /tmp] echo UEsFBgAAAAAAAAAAAAAAAAAAAAAAAA== | base64 -d > grenoble_2018-12-20_1829997.zip
[2018-12-20 17:43:34][DEBUG] [direct

## Gathering a trace with optimization level 1

In [3]:
def build_hpl(job, opt_level):
    job.nodes.run('git clean -xfd .', directory=job.hpl_dir)
    options = '-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=%d' % opt_level
    while True:
        try:
            job.nodes.run('make SMPI_OPTS="%s" arch=SMPI' % options, directory=job.hpl_dir)
        except RunError as e:  # for some reason, this command fails sometime...
            msg = str(e).split('\n')[0]
            logger.error('Previous command failed with message %s' % msg)
        else:
            break

In [4]:
build_hpl(job, 1)

[2018-12-20 17:47:23][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 17:47:23][DEBUG] [allnodes | /tmp/hpl-2.2] make SMPI_OPTS="-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=1" arch=SMPI


Let's take the experiment files I used previously.

In [5]:
!unzip -p grenoble_2018-12-18_1829370.zip exp_smpi.csv > /tmp/exp_smpi.csv
!unzip -p grenoble_2018-12-18_1829370.zip dahu.xml > /tmp/dahu.xml
job.expfile = [ExpFile(filename='/tmp/exp_smpi.csv', types=job.expfile_types, header=job.expfile_header, header_in_file=job.expfile_header_in_file),
               ExpFile(filename='/tmp/dahu.xml')]

In [6]:
job.expfile[0].content

[{'matrix_size': 50000,
  'block_size': 128,
  'proc_p': 2,
  'proc_q': 4,
  'pfact': 1,
  'rfact': 2,
  'bcast': 2,
  'depth': 1,
  'swap': 0,
  'mem_align': 8,
  'process_per_node': 1,
  'thread_per_process': 32,
  'dgemm_coefficient': 2.548998e-12,
  'dgemm_intercept': 9.905042e-05,
  'dtrsm_coefficient': 4.524788e-12,
  'dtrsm_intercept': 3.975362e-05}]

In [7]:
print(job.expfile[1].raw_content)

<?xml version="1.0"?>
<!DOCTYPE platform SYSTEM "http://simgrid.gforge.inria.fr/simgrid/simgrid.dtd">

<platform version="4.1">
    <config id="General">
        <prop id="smpi/os" value="0:2.9654918472829634e-07:9.636195922971908e-11;8133:0.0:0.0;15831:0.0:0.0;33956:0.0:0.0;63305:0.0:0.0"/>
        <prop id="smpi/or" value="0:1.3754300044375224e-06:8.456019002037343e-11;8133:0.0:0.0;15831:0.0:0.0;33956:0.0:0.0;63305:0.0:0.0"/>
        <prop id="smpi/ois" value="0:6.93122952108322e-07:7.051204536228499e-11;8133:3.601990790153314e-07:1.2449128895712003e-10;15831:2.977016881030904e-06:4.130261264964205e-11;33956:3.1334661540668004e-06:3.293458765282178e-11;63305:6.939993663604218e-07:0.0"/>
        <prop id="smpi/bw-factor" value="0:0.4709912642540426;8133:0.6770561715947873;15831:0.7091971477925826;33956:0.7033449540683548;63305:0.9867951082730284"/>
        <prop id="smpi/lat-factor" value="0:1.1347371681807956;8133:29.84698135078078;15831:30.274775156187765;33956:64.63917831305292;633

In [8]:
job.run_exp()

[2018-12-20 17:47:41][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpt61vm0du → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 17:47:41][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpoxcp00t9 → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 17:47:41][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpyfo3dwsl → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 17:47:42][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 17:47:42][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp_8t1a0m5 → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 17:47:42][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [9]:
def get_trace(job, functions, ranks):
    func_reg = '|'.join(functions)
    rank_reg = '|'.join(['rank-%d' % r for r in ranks])
    cmd = 'grep -E "%s" /tmp/trace_mpi_0.csv | grep -E "%s" > /tmp/mpi.csv' % (func_reg, rank_reg)
    job.nodes.run(cmd)
    job.nodes.get('/tmp/mpi.csv', '/tmp/mpi.csv')
    df = pandas.read_csv('/tmp/mpi.csv', names=['type', 'rank', 'container', 'start', 'end', 'duration', 'level', 'function', 'msg_size'])
    df['function'] = df['function'].str.replace('PMPI', 'MPI').str.strip()
    df['rank'] = df['rank'].str.slice(6).astype(int)  # changing 'rank-42' into 42
    return df.sort_values(by=['rank', 'start'])

trace_level1 = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_level1.head()

[2018-12-20 17:53:35][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 17:53:38][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
188122,State,0,MPI_STATE,0.000062,0.000063,0.000001,0.0,MPI_Recv,4
188123,State,0,MPI_STATE,0.000064,0.000065,0.000001,0.0,MPI_Recv,4
188124,State,0,MPI_STATE,0.000065,0.000067,0.000002,0.0,MPI_Recv,4
188125,State,0,MPI_STATE,0.000069,0.000070,0.000001,0.0,MPI_Send,4
188126,State,0,MPI_STATE,0.000070,0.000070,0.000000,0.0,MPI_Send,4


In [10]:
trace_level1.groupby(['function', 'rank'])[['msg_size']].agg(['count', 'sum'])

msg_size            
                  count         sum
function  rank                     
MPI_Irecv 0       13028  1881623584
          1       13030  1900856720
          2       12984  1886700272
          3       12899  1881537208
          4       13028  1895193232
          5       13030  1895144152
          6       12984  1900204880
          7       12899  1875819768
MPI_Recv  0         421  3776632664
          1         505  3775431788
          2         416  3788906148
          3         504  3788355700
          4         516  3765221548
          5         405  3776868460
          6         510  3777548044
          7         405  3789713524
MPI_Send  0       13557  5683351676
          1       13431  5683851192
          2       13501  5676089100
          3       13298  5651597776
          4       13455  5684383112
          5       13527  5665759960
          6       13401  5677133588
          7       13394  5633591288

In [11]:
def trace_diff(old_trace, new_trace):
    old_summary = old_trace.groupby(['function', 'rank'])[['msg_size']].agg(['count', 'sum'])
    new_summary = new_trace.groupby(['function', 'rank'])[['msg_size']].agg(['count', 'sum'])
    return (new_summary-old_summary)/old_summary

In [12]:
trace_diff(trace_level1, trace_level1)

msg_size     
                  count  sum
function  rank              
MPI_Irecv 0         0.0  0.0
          1         0.0  0.0
          2         0.0  0.0
          3         0.0  0.0
          4         0.0  0.0
          5         0.0  0.0
          6         0.0  0.0
          7         0.0  0.0
MPI_Recv  0         0.0  0.0
          1         0.0  0.0
          2         0.0  0.0
          3         0.0  0.0
          4         0.0  0.0
          5         0.0  0.0
          6         0.0  0.0
          7         0.0  0.0
MPI_Send  0         0.0  0.0
          1         0.0  0.0
          2         0.0  0.0
          3         0.0  0.0
          4         0.0  0.0
          5         0.0  0.0
          6         0.0  0.0
          7         0.0  0.0

## Gathering a trace with optimization level 2

In [13]:
build_hpl(job, 2)

[2018-12-20 17:53:41][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 17:53:41][DEBUG] [allnodes | /tmp/hpl-2.2] make SMPI_OPTS="-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=2" arch=SMPI


In [14]:
job.run_exp()

[2018-12-20 17:53:58][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp3m4lpl37 → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 17:53:58][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp1h8nxfns → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 17:53:58][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp97xw1mtg → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 17:53:58][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 17:53:58][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpy83ik8xv → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 17:53:59][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [15]:
trace_level2 = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_level2.head()

[2018-12-20 17:54:58][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 17:55:01][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
184199,State,0,MPI_STATE,0.000090,0.000091,0.000001,0.0,MPI_Recv,4
184200,State,0,MPI_STATE,0.000093,0.000094,0.000001,0.0,MPI_Recv,4
184201,State,0,MPI_STATE,0.000094,0.000095,0.000001,0.0,MPI_Recv,4
184202,State,0,MPI_STATE,0.000099,0.000100,0.000001,0.0,MPI_Send,4
184203,State,0,MPI_STATE,0.000100,0.000100,0.000000,0.0,MPI_Send,4


In [16]:
trace_level2.groupby(['function', 'rank'])[['msg_size']].agg(['count', 'sum'])

msg_size            
                  count         sum
function  rank                     
MPI_Irecv 0       12834  1259216896
          1       12738  1272061952
          2       12789  1262595584
          3       12609  1259081728
          4       12834    35827984
          5       12738    22982160
          6       12789    35693760
          7       12609    22747656
MPI_Recv  0         305  3775030280
          1         301  3775030332
          2         305  3787906524
          3         302  3787958340
          4         503  5009794172
          5         593  5035484220
          6         501  5026180188
          7         592  5035485252
MPI_Send  0       13346  5069757052
          1       13327  5069704024
          2       13291  5060207140
          3       13195  5043696456
          4       13148  5061576952
          5       13035  5035965768
          6       13095  5052030180
          7       12905  5010139456

In [17]:
trace_diff(trace_level1, trace_level2)

msg_size          
                   count       sum
function  rank                    
MPI_Irecv 0    -0.014891 -0.330782
          1    -0.022410 -0.330795
          2    -0.015018 -0.330792
          3    -0.022482 -0.330823
          4    -0.014891 -0.981095
          5    -0.022410 -0.987873
          6    -0.015018 -0.981216
          7    -0.022482 -0.987873
MPI_Recv  0    -0.275534 -0.000424
          1    -0.403960 -0.000106
          2    -0.266827 -0.000264
          3    -0.400794 -0.000105
          4    -0.025194  0.330544
          5     0.464198  0.333243
          6    -0.017647  0.330540
          7     0.461728  0.328725
MPI_Send  0    -0.015564 -0.107964
          1    -0.007743 -0.108051
          2    -0.015554 -0.108505
          3    -0.007746 -0.107563
          4    -0.022817 -0.109564
          5    -0.036372 -0.111158
          6    -0.022834 -0.110109
          7    -0.036509 -0.110667

## Patching my version of HPL

First, let's get the original source code.

In [18]:
job.nodes.run('wget http://www.netlib.org/benchmark/hpl/hpl-2.2.tar.gz')
job.nodes.run('mkdir original')
job.nodes.run('tar -xvf hpl-2.2.tar.gz -C original')

[2018-12-20 17:55:04][DEBUG] [allnodes | /tmp] wget http://www.netlib.org/benchmark/hpl/hpl-2.2.tar.gz
[2018-12-20 17:55:04][DEBUG] [allnodes | /tmp] mkdir original
[2018-12-20 17:55:05][DEBUG] [allnodes | /tmp] tar -xvf hpl-2.2.tar.gz -C original


{<Connection host=dahu-9.grenoble.grid5000.fr user=root gw=proxyjump>: <Result cmd='cd /tmp && tar -xvf hpl-2.2.tar.gz -C original' exited=0>}

In [19]:
def change_hpl_files(job, filenames, original_dir='/tmp/original/hpl-2.2'):
    for file in filenames:
        job.nodes.run('cp -f %s/%s %s/%s' % (original_dir, file, job.hpl_dir, file))

In [20]:
def remove_hpl_lines(job, filename, lines):
    lines = ';'.join(['%dd' % l for l in lines])
    job.nodes.run('sed -i -e "%s" %s/%s' % (lines, job.hpl_dir, filename))

In [21]:
def patch_hpl(job, *patches):
    job.nodes.run('git checkout src include testing', directory=job.hpl_dir)
    job.nodes.run('git clean -xfd .', directory=job.hpl_dir)
    for func, *args in patches:
        func(job, *args)
    build_hpl(job, 2)
    print(job.nodes.run_unique('git diff --color src include testing', directory=job.hpl_dir).stdout)

### HPL utility functions

In [22]:
files = [
    'src/auxil/HPL_dlatcpy.c',
    'src/auxil/HPL_dlacpy.c',
    'src/pauxil/HPL_dlaswp01T.c',
    'src/pauxil/HPL_dlaswp06T.c',
    'src/pauxil/HPL_dlaswp00N.c',
    'src/pauxil/HPL_dlaswp05N.c',
    'src/pauxil/HPL_dlaswp03T.c',
    'src/pauxil/HPL_dlaswp10N.c',
    'src/pauxil/HPL_dlaswp05T.c',
    'src/pauxil/HPL_dlaswp02N.c',
    'src/pauxil/HPL_dlaswp03N.c',
    'src/pauxil/HPL_dlaswp01N.c',
    'src/pauxil/HPL_dlaswp04N.c',
    'src/pauxil/HPL_dlaswp04T.c',
    'src/pauxil/HPL_dlaswp06N.c',
]
patch_utility = (change_hpl_files, files)
patch_hpl(job, patch_utility)

[2018-12-20 17:55:05][DEBUG] [allnodes | /tmp/hpl-2.2] git checkout src include testing
[2018-12-20 17:55:05][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 17:55:05][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/auxil/HPL_dlatcpy.c hpl-2.2/src/auxil/HPL_dlatcpy.c
[2018-12-20 17:55:05][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/auxil/HPL_dlacpy.c hpl-2.2/src/auxil/HPL_dlacpy.c
[2018-12-20 17:55:06][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/pauxil/HPL_dlaswp01T.c hpl-2.2/src/pauxil/HPL_dlaswp01T.c
[2018-12-20 17:55:06][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/pauxil/HPL_dlaswp06T.c hpl-2.2/src/pauxil/HPL_dlaswp06T.c
[2018-12-20 17:55:06][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/pauxil/HPL_dlaswp00N.c hpl-2.2/src/pauxil/HPL_dlaswp00N.c
[2018-12-20 17:55:06][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/pauxil/HPL_dlaswp05N.c hpl-2.2/src/pauxil/HPL_dlaswp05N.c
[2018-12-20 17:55:07][DEBUG] [al

diff --git a/src/auxil/HPL_dlacpy.c b/src/auxil/HPL_dlacpy.c
index 8c1396a..89ae13b 100644
--- a/src/auxil/HPL_dlacpy.c
+++ b/src/auxil/HPL_dlacpy.c
@@ -127,9 +127,6 @@ void HPL_dlacpy
 /*
  * .. Local Variables ..
  */
-#if SMPI_OPTIMIZATION_LEVEL >= 2
-    return;
-#endif
 #ifdef HPL_LACPY_USE_COPY
    register int               j;
 #else
diff --git a/src/auxil/HPL_dlatcpy.c b/src/auxil/HPL_dlatcpy.c
index 417a1d5..7643676 100644
--- a/src/auxil/HPL_dlatcpy.c
+++ b/src/auxil/HPL_dlatcpy.c
@@ -127,9 +127,6 @@ void HPL_dlatcpy
 /*
  * .. Local Variables ..
  */
-#if SMPI_OPTIMIZATION_LEVEL >= 2
-    return;
-#endif
 #ifdef HPL_LATCPY_USE_COPY
    register int               j;
 #else
diff --git a/src/pauxil/HPL_dlaswp00N.c b/src/pauxil/HPL_dlaswp00N.c
index 2cab3c3..60ae8b1 100644
--- a/src/pauxil/HPL_dlaswp00N.c
+++ b/src/pauxil/HPL_dlaswp00N.c
@@ -112,9 +112,6 @@ void HPL_dlaswp00N
 /*
  * .. Local Variables ..
  */
-#if SMPI_OPTIMIZATION_LEVEL >= 2
-    return;
-#endif
    register d

In [23]:
job.run_exp()

[2018-12-20 17:55:26][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpxdhmqdrv → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 17:55:27][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp3qqnc1qo → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 17:55:27][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpdipxg5ar → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 17:55:27][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 17:55:27][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpexasa1cq → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 17:55:27][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [24]:
trace_tmp = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_tmp.head()

[2018-12-20 17:56:33][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 17:56:35][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
184199,State,0,MPI_STATE,0.000064,0.000066,0.000002,0.0,MPI_Recv,4
184200,State,0,MPI_STATE,0.000066,0.000067,0.000001,0.0,MPI_Recv,4
184201,State,0,MPI_STATE,0.000067,0.000069,0.000002,0.0,MPI_Recv,4
184202,State,0,MPI_STATE,0.000071,0.000071,0.000000,0.0,MPI_Send,4
184203,State,0,MPI_STATE,0.000071,0.000072,0.000001,0.0,MPI_Send,4


In [25]:
trace_diff(trace_level1, trace_tmp)

msg_size          
                   count       sum
function  rank                    
MPI_Irecv 0    -0.014891 -0.330782
          1    -0.022410 -0.330795
          2    -0.015018 -0.330792
          3    -0.022482 -0.330823
          4    -0.014891 -0.981095
          5    -0.022410 -0.987873
          6    -0.015018 -0.981216
          7    -0.022482 -0.987873
MPI_Recv  0    -0.275534 -0.000424
          1    -0.403960 -0.000106
          2    -0.266827 -0.000264
          3    -0.400794 -0.000105
          4    -0.025194  0.330544
          5     0.464198  0.333243
          6    -0.017647  0.330540
          7     0.461728  0.328725
MPI_Send  0    -0.015564 -0.107964
          1    -0.007743 -0.108051
          2    -0.015554 -0.108505
          3    -0.007746 -0.107563
          4    -0.022817 -0.109564
          5    -0.036372 -0.111158
          6    -0.022834 -0.110109
          7    -0.036509 -0.110667

Same values than for the trace with level 2, so the HPL functions are probably not the culprit.

### BLAS functions

In [26]:
patch_blas = (remove_hpl_lines, 'include/hpl_blas.h', list(range(164, 174))+[182])
patch_hpl(job, patch_blas)

[2018-12-20 17:56:38][DEBUG] [allnodes | /tmp/hpl-2.2] git checkout src include testing
[2018-12-20 17:56:38][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 17:56:38][DEBUG] [allnodes | /tmp] sed -i -e "164d;165d;166d;167d;168d;169d;170d;171d;172d;173d;182d" hpl-2.2/include/hpl_blas.h
[2018-12-20 17:56:39][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 17:56:39][DEBUG] [allnodes | /tmp/hpl-2.2] make SMPI_OPTS="-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=2" arch=SMPI
[2018-12-20 17:56:55][DEBUG] [allnodes | /tmp/hpl-2.2] git diff --color src include testing


diff --git a/include/hpl_blas.h b/include/hpl_blas.h
index 023ec77..eecece4 100644
--- a/include/hpl_blas.h
+++ b/include/hpl_blas.h
@@ -161,16 +161,6 @@ STDC_ARGS(
  */
 #define    HPL_dcopy           cblas_dcopy
 
-#if SMPI_OPTIMIZATION_LEVEL >= 2
-#define    HPL_dswap(...)      {}
-#define    HPL_dgemv(...)      {}
-#define    HPL_daxpy(...)      {}
-#define    HPL_dscal(...)      {}
-#define    HPL_idamax(...)      3 // FIXME: 3 is arbitrary... is the value returned by idamax important?
-#define    HPL_dtrsv(...)      {}
-#define    HPL_dger(...)       {}
-#pragma message "[SMPI] Using no-op for the “cheapest” BLAS functions."
-#else
 #define    HPL_dswap           cblas_dswap
 #define    HPL_dgemv           cblas_dgemv
 #define    HPL_daxpy           cblas_daxpy
@@ -179,7 +169,6 @@ STDC_ARGS(
 #define    HPL_dtrsv           cblas_dtrsv
 #define    HPL_dger            cblas_dger
 #pragma message "[SMPI] Using cblas for the “cheapest” BLAS functions."
-#endif
 
 // From http://stack

In [27]:
job.run_exp()

[2018-12-20 17:56:56][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp4i3x71p0 → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 17:56:56][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpg9r09da_ → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 17:56:56][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmptj0aeq1p → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 17:56:56][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 17:56:56][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpnqp0896k → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 17:56:57][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [28]:
trace_tmp = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_tmp.head()

[2018-12-20 17:58:33][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 17:58:35][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
184199,State,0,MPI_STATE,0.000066,0.000067,0.000001,0.0,MPI_Recv,4
184200,State,0,MPI_STATE,0.000069,0.000070,0.000001,0.0,MPI_Recv,4
184201,State,0,MPI_STATE,0.000070,0.000071,0.000001,0.0,MPI_Recv,4
184202,State,0,MPI_STATE,0.000074,0.000074,0.000000,0.0,MPI_Send,4
184203,State,0,MPI_STATE,0.000074,0.000074,0.000000,0.0,MPI_Send,4


In [29]:
trace_diff(trace_level1, trace_tmp)

msg_size          
                   count       sum
function  rank                    
MPI_Irecv 0    -0.014891 -0.330782
          1    -0.022410 -0.330795
          2    -0.015018 -0.330792
          3    -0.022482 -0.330823
          4    -0.014891 -0.981095
          5    -0.022410 -0.987873
          6    -0.015018 -0.981216
          7    -0.022482 -0.987873
MPI_Recv  0    -0.275534 -0.000424
          1    -0.403960 -0.000106
          2    -0.266827 -0.000264
          3    -0.400794 -0.000105
          4    -0.025194  0.330544
          5     0.464198  0.333243
          6    -0.017647  0.330540
          7     0.461728  0.328725
MPI_Send  0    -0.015564 -0.107964
          1    -0.007743 -0.108051
          2    -0.015554 -0.108505
          3    -0.007746 -0.107563
          4    -0.022817 -0.109564
          5    -0.036372 -0.111158
          6    -0.022834 -0.110109
          7    -0.036509 -0.110667

### Initialization and verification

In [30]:
patch_init = (remove_hpl_lines, 'testing/ptest/HPL_pdtest.c', [63, 64, 65, 202, 204, 338, 339, 340, 341, 342])
patch_hpl(job, patch_init)

[2018-12-20 17:58:38][DEBUG] [allnodes | /tmp/hpl-2.2] git checkout src include testing
[2018-12-20 17:58:38][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 17:58:38][DEBUG] [allnodes | /tmp] sed -i -e "63d;64d;65d;202d;204d;338d;339d;340d;341d;342d" hpl-2.2/testing/ptest/HPL_pdtest.c
[2018-12-20 17:58:39][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 17:58:39][DEBUG] [allnodes | /tmp/hpl-2.2] make SMPI_OPTS="-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=2" arch=SMPI
[2018-12-20 17:58:56][DEBUG] [allnodes | /tmp/hpl-2.2] git diff --color src include testing


diff --git a/testing/ptest/HPL_pdtest.c b/testing/ptest/HPL_pdtest.c
index 5166be3..109d489 100644
--- a/testing/ptest/HPL_pdtest.c
+++ b/testing/ptest/HPL_pdtest.c
@@ -60,9 +60,6 @@
 #define smpi_free free
 #endif
 
-#if SMPI_OPTIMIZATION_LEVEL <= 1
-#define SMPI_DO_INITIALIZATION_VERIFICATION
-#endif
 
 #ifdef STDC_HEADERS
 void HPL_pdtest
@@ -199,9 +196,7 @@ void HPL_pdtest
    mat.A  = (double *)HPL_PTR( vptr,
                                ((size_t)(ALGO->align) * sizeof(double) ) );
    mat.X  = Mptr( mat.A, 0, mat.nq, mat.ld );
-#ifdef SMPI_DO_INITIALIZATION_VERIFICATION
    HPL_pdmatgen( GRID, N, N+1, NB, mat.A, mat.ld, HPL_ISEED );
-#endif
 #ifdef HPL_CALL_VSIPL
    mat.block = vsip_blockbind_d( (vsip_scalar_d *)(mat.A),
                                  (vsip_length)(mat.ld * mat.nq),
@@ -335,11 +330,6 @@ void HPL_pdtest
    }
 #endif
 
-#ifndef SMPI_DO_INITIALIZATION_VERIFICATION
-    if(vptr)
-        smpi_free(vptr);
-    return;
-#endif
 /*
  * Quick return, if I am not 

In [31]:
job.run_exp()

[2018-12-20 17:58:56][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpn8p00zgp → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 17:58:56][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpf7t8ituq → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 17:58:56][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpln7o89oe → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 17:58:57][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 17:58:57][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmppb_i2_db → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 17:58:57][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [32]:
trace_tmp = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_tmp.head()

[2018-12-20 18:02:02][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 18:02:05][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
188122,State,0,MPI_STATE,0.000079,0.000080,0.000001,0.0,MPI_Recv,4
188123,State,0,MPI_STATE,0.000082,0.000083,0.000001,0.0,MPI_Recv,4
188124,State,0,MPI_STATE,0.000083,0.000085,0.000002,0.0,MPI_Recv,4
188125,State,0,MPI_STATE,0.000088,0.000088,0.000000,0.0,MPI_Send,4
188126,State,0,MPI_STATE,0.000088,0.000089,0.000001,0.0,MPI_Send,4


In [33]:
trace_diff(trace_level1, trace_tmp)

msg_size          
                  count       sum
function  rank                   
MPI_Irecv 0         0.0 -0.325608
          1         0.0 -0.325621
          2         0.0 -0.325617
          3         0.0 -0.325648
          4         0.0 -0.323658
          5         0.0 -0.323641
          6         0.0 -0.323692
          7         0.0 -0.323646
MPI_Recv  0         0.0  0.000000
          1         0.0  0.000000
          2         0.0  0.000000
          3         0.0  0.000000
          4         0.0  0.000000
          5         0.0  0.000000
          6         0.0  0.000000
          7         0.0  0.000000
MPI_Send  0         0.0 -0.107928
          1         0.0 -0.107910
          2         0.0 -0.108363
          3         0.0 -0.107421
          4         0.0 -0.107781
          5         0.0 -0.109245
          6         0.0 -0.108213
          7         0.0 -0.108762

### Initialization and verification & BLAS

In [34]:
patch_hpl(job, patch_init, patch_blas)

[2018-12-20 18:02:07][DEBUG] [allnodes | /tmp/hpl-2.2] git checkout src include testing
[2018-12-20 18:02:07][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 18:02:08][DEBUG] [allnodes | /tmp] sed -i -e "63d;64d;65d;202d;204d;338d;339d;340d;341d;342d" hpl-2.2/testing/ptest/HPL_pdtest.c
[2018-12-20 18:02:08][DEBUG] [allnodes | /tmp] sed -i -e "164d;165d;166d;167d;168d;169d;170d;171d;172d;173d;182d" hpl-2.2/include/hpl_blas.h
[2018-12-20 18:02:08][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 18:02:08][DEBUG] [allnodes | /tmp/hpl-2.2] make SMPI_OPTS="-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=2" arch=SMPI
[2018-12-20 18:02:25][DEBUG] [allnodes | /tmp/hpl-2.2] git diff --color src include testing


diff --git a/include/hpl_blas.h b/include/hpl_blas.h
index 023ec77..eecece4 100644
--- a/include/hpl_blas.h
+++ b/include/hpl_blas.h
@@ -161,16 +161,6 @@ STDC_ARGS(
  */
 #define    HPL_dcopy           cblas_dcopy
 
-#if SMPI_OPTIMIZATION_LEVEL >= 2
-#define    HPL_dswap(...)      {}
-#define    HPL_dgemv(...)      {}
-#define    HPL_daxpy(...)      {}
-#define    HPL_dscal(...)      {}
-#define    HPL_idamax(...)      3 // FIXME: 3 is arbitrary... is the value returned by idamax important?
-#define    HPL_dtrsv(...)      {}
-#define    HPL_dger(...)       {}
-#pragma message "[SMPI] Using no-op for the “cheapest” BLAS functions."
-#else
 #define    HPL_dswap           cblas_dswap
 #define    HPL_dgemv           cblas_dgemv
 #define    HPL_daxpy           cblas_daxpy
@@ -179,7 +169,6 @@ STDC_ARGS(
 #define    HPL_dtrsv           cblas_dtrsv
 #define    HPL_dger            cblas_dger
 #pragma message "[SMPI] Using cblas for the “cheapest” BLAS functions."
-#endif
 
 // From http://stack

In [35]:
job.run_exp()

[2018-12-20 18:02:25][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmph5c9abl5 → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 18:02:25][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp19bw57v9 → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 18:02:26][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpz09g91pw → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 18:02:26][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 18:02:26][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpa77nat7e → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 18:02:26][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [36]:
trace_tmp = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_tmp.head()

[2018-12-20 18:06:20][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 18:06:23][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
188122,State,0,MPI_STATE,0.000095,0.000096,0.000001,0.0,MPI_Recv,4
188123,State,0,MPI_STATE,0.000098,0.000099,0.000001,0.0,MPI_Recv,4
188124,State,0,MPI_STATE,0.000099,0.000101,0.000002,0.0,MPI_Recv,4
188125,State,0,MPI_STATE,0.000104,0.000105,0.000001,0.0,MPI_Send,4
188126,State,0,MPI_STATE,0.000105,0.000105,0.000000,0.0,MPI_Send,4


In [37]:
trace_diff(trace_level1, trace_tmp)

msg_size          
                  count       sum
function  rank                   
MPI_Irecv 0         0.0 -0.003723
          1         0.0 -0.003691
          2         0.0 -0.003711
          3         0.0 -0.003723
          4         0.0  0.000030
          5         0.0  0.000030
          6         0.0  0.000058
          7         0.0  0.000056
MPI_Recv  0         0.0  0.000000
          1         0.0  0.000000
          2         0.0  0.000000
          3         0.0  0.000000
          4         0.0  0.000000
          5         0.0  0.000000
          6         0.0  0.000000
          7         0.0  0.000000
MPI_Send  0         0.0  0.000010
          1         0.0  0.000010
          2         0.0  0.000019
          3         0.0  0.000019
          4         0.0 -0.001232
          5         0.0 -0.001238
          6         0.0 -0.001233
          7         0.0 -0.001244

### Initialization and verification & BLAS & utility

In [38]:
patch_hpl(job, patch_init, patch_blas, patch_utility)

[2018-12-20 18:06:26][DEBUG] [allnodes | /tmp/hpl-2.2] git checkout src include testing
[2018-12-20 18:06:26][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 18:06:26][DEBUG] [allnodes | /tmp] sed -i -e "63d;64d;65d;202d;204d;338d;339d;340d;341d;342d" hpl-2.2/testing/ptest/HPL_pdtest.c
[2018-12-20 18:06:26][DEBUG] [allnodes | /tmp] sed -i -e "164d;165d;166d;167d;168d;169d;170d;171d;172d;173d;182d" hpl-2.2/include/hpl_blas.h
[2018-12-20 18:06:27][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/auxil/HPL_dlatcpy.c hpl-2.2/src/auxil/HPL_dlatcpy.c
[2018-12-20 18:06:27][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/auxil/HPL_dlacpy.c hpl-2.2/src/auxil/HPL_dlacpy.c
[2018-12-20 18:06:27][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/pauxil/HPL_dlaswp01T.c hpl-2.2/src/pauxil/HPL_dlaswp01T.c
[2018-12-20 18:06:27][DEBUG] [allnodes | /tmp] cp -f /tmp/original/hpl-2.2/src/pauxil/HPL_dlaswp06T.c hpl-2.2/src/pauxil/HPL_dlaswp06T.c
[2018-12-20 18:06:27][DE

diff --git a/include/hpl_blas.h b/include/hpl_blas.h
index 023ec77..eecece4 100644
--- a/include/hpl_blas.h
+++ b/include/hpl_blas.h
@@ -161,16 +161,6 @@ STDC_ARGS(
  */
 #define    HPL_dcopy           cblas_dcopy
 
-#if SMPI_OPTIMIZATION_LEVEL >= 2
-#define    HPL_dswap(...)      {}
-#define    HPL_dgemv(...)      {}
-#define    HPL_daxpy(...)      {}
-#define    HPL_dscal(...)      {}
-#define    HPL_idamax(...)      3 // FIXME: 3 is arbitrary... is the value returned by idamax important?
-#define    HPL_dtrsv(...)      {}
-#define    HPL_dger(...)       {}
-#pragma message "[SMPI] Using no-op for the “cheapest” BLAS functions."
-#else
 #define    HPL_dswap           cblas_dswap
 #define    HPL_dgemv           cblas_dgemv
 #define    HPL_daxpy           cblas_daxpy
@@ -179,7 +169,6 @@ STDC_ARGS(
 #define    HPL_dtrsv           cblas_dtrsv
 #define    HPL_dger            cblas_dger
 #pragma message "[SMPI] Using cblas for the “cheapest” BLAS functions."
-#endif
 
 // From http://stack

In [39]:
job.run_exp()

[2018-12-20 18:06:47][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpojduh2kd → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 18:06:48][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp2qlxim1b → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 18:06:48][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp9j3rovg0 → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 18:06:48][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 18:06:48][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpontoawun → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 18:06:49][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [40]:
trace_tmp = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_tmp.head()

[2018-12-20 18:12:05][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 18:12:07][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
188122,State,0,MPI_STATE,0.000087,0.000088,0.000001,0.0,MPI_Recv,4
188123,State,0,MPI_STATE,0.000090,0.000091,0.000001,0.0,MPI_Recv,4
188124,State,0,MPI_STATE,0.000091,0.000092,0.000001,0.0,MPI_Recv,4
188125,State,0,MPI_STATE,0.000096,0.000096,0.000000,0.0,MPI_Send,4
188126,State,0,MPI_STATE,0.000096,0.000097,0.000001,0.0,MPI_Send,4


In [41]:
trace_diff(trace_level1, trace_tmp)

msg_size     
                  count  sum
function  rank              
MPI_Irecv 0         0.0  0.0
          1         0.0  0.0
          2         0.0  0.0
          3         0.0  0.0
          4         0.0  0.0
          5         0.0  0.0
          6         0.0  0.0
          7         0.0  0.0
MPI_Recv  0         0.0  0.0
          1         0.0  0.0
          2         0.0  0.0
          3         0.0  0.0
          4         0.0  0.0
          5         0.0  0.0
          6         0.0  0.0
          7         0.0  0.0
MPI_Send  0         0.0  0.0
          1         0.0  0.0
          2         0.0  0.0
          3         0.0  0.0
          4         0.0  0.0
          5         0.0  0.0
          6         0.0  0.0
          7         0.0  0.0

### Same thing, with optimization level 3

In [42]:
build_hpl(job, 3)

[2018-12-20 18:12:10][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 18:12:10][DEBUG] [allnodes | /tmp/hpl-2.2] make SMPI_OPTS="-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=3" arch=SMPI


In [43]:
job.run_exp()

[2018-12-20 18:12:28][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpadq8e85w → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 18:12:28][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpwcx915jc → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 18:12:28][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpxd_6et2l → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 18:12:29][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 18:12:29][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpwtnean9d → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 18:12:29][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [44]:
trace_tmp = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_tmp.head()

[2018-12-20 18:15:05][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 18:15:08][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
185758,State,0,MPI_STATE,0.000066,0.000067,0.000001,0.0,MPI_Recv,4
185759,State,0,MPI_STATE,0.000068,0.000070,0.000002,0.0,MPI_Recv,4
185760,State,0,MPI_STATE,0.000070,0.000071,0.000001,0.0,MPI_Recv,4
185761,State,0,MPI_STATE,0.000073,0.000074,0.000001,0.0,MPI_Send,4
185762,State,0,MPI_STATE,0.000074,0.000074,0.000000,0.0,MPI_Send,4


In [45]:
trace_diff(trace_level1, trace_tmp)

msg_size          
                   count       sum
function  rank                    
MPI_Irecv 0    -0.001612 -0.010378
          1    -0.002379 -0.010919
          2    -0.001617 -0.010474
          3    -0.002481 -0.010384
          4    -0.001612  0.097776
          5    -0.002379  0.097076
          6    -0.001617  0.097718
          7    -0.002481  0.097328
MPI_Recv  0    -0.275534 -0.000424
          1    -0.403960 -0.000106
          2    -0.266827 -0.000264
          3    -0.400794 -0.000105
          4    -0.360465  0.016129
          5    -0.180247  0.016744
          6    -0.358824  0.016315
          7    -0.175309  0.016341
MPI_Send  0    -0.015564  0.043501
          1    -0.007743  0.043388
          2    -0.015554  0.043571
          3    -0.007746  0.043156
          4    -0.009959 -0.003506
          5    -0.017077 -0.003840
          6    -0.009850 -0.003657
          7    -0.017247 -0.003645

### Same thing, with optimization level 4

In [46]:
build_hpl(job, 4)

[2018-12-20 18:15:11][DEBUG] [allnodes | /tmp/hpl-2.2] git clean -xfd .
[2018-12-20 18:15:11][DEBUG] [allnodes | /tmp/hpl-2.2] make SMPI_OPTS="-DSMPI_MEASURE -DSMPI_OPTIMIZATION_LEVEL=4" arch=SMPI


In [47]:
job.run_exp()

[2018-12-20 18:15:28][DEBUG] [director] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpv688r3ls → /tmp/hpl-2.2/bin/SMPI/concatenate.sh
[2018-12-20 18:15:29][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpputblkd9 → /tmp/hpl-2.2/bin/SMPI/platform.xml
[2018-12-20 18:15:29][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmp_7jc3u13 → /tmp/hpl-2.2/bin/SMPI/hosts.txt
[2018-12-20 18:15:29][WARNING] Requested 1 process per node and 32 thread per process, but 1 cores are available
[2018-12-20 18:15:29][DEBUG] [allnodes] put: /home/tom/Dropbox/Documents/Fac/phd/mpi_calibration/dahu/smpi_hpl/tmpl7jmmfws → /tmp/hpl-2.2/bin/SMPI/HPL.dat
[2018-12-20 18:15:29][DEBUG] [director | /tmp/hpl-2.2/bin/SMPI] SMPI_DGEMM_COEFFICIENT=2.548998e-12 SMPI_DGEMM_INTERCEPT=9.905042e-05 SMPI_DTRSM_COEFFICIENT=4.524788e-12 SMPI_DTRSM_INTERCEPT=3.975362e-05 TIME="/usr/bin/time:output %U %S %F %R %P" LD_LIB

In [48]:
trace_tmp = get_trace(job, ['MPI_Recv', 'MPI_Send', 'MPI_Irecv'], range(8))
trace_tmp.head()

[2018-12-20 18:18:00][DEBUG] [allnodes | /tmp] grep -E "MPI_Recv|MPI_Send|MPI_Irecv" /tmp/trace_mpi_0.csv | grep -E "rank-0|rank-1|rank-2|rank-3|rank-4|rank-5|rank-6|rank-7" > /tmp/mpi.csv
[2018-12-20 18:18:03][DEBUG] [allnodes] get: /tmp/mpi.csv → /tmp/mpi.csv


,type,rank,container,start,end,duration,level,function,msg_size
185587,State,0,MPI_STATE,0.000063,0.000064,0.000001,0.0,MPI_Recv,4
185588,State,0,MPI_STATE,0.000065,0.000066,0.000001,0.0,MPI_Recv,4
185589,State,0,MPI_STATE,0.000066,0.000068,0.000002,0.0,MPI_Recv,4
185590,State,0,MPI_STATE,0.000070,0.000071,0.000001,0.0,MPI_Send,4
185591,State,0,MPI_STATE,0.000071,0.000071,0.000000,0.0,MPI_Send,4


In [49]:
trace_diff(trace_level1, trace_tmp)

msg_size          
                   count       sum
function  rank                    
MPI_Irecv 0    -0.003070 -0.167767
          1    -0.004451 -0.167581
          2    -0.003081 -0.167669
          3    -0.004807 -0.167787
          4    -0.003070  0.026653
          5    -0.004451  0.025395
          6    -0.003081  0.026760
          7    -0.004807  0.025215
MPI_Recv  0    -0.275534 -0.000424
          1    -0.403960 -0.000106
          2    -0.266827 -0.000264
          3    -0.400794 -0.000105
          4    -0.323643  0.087005
          5    -0.113580  0.087679
          6    -0.321569  0.086995
          7    -0.101235  0.086759
MPI_Send  0    -0.015564  0.066739
          1    -0.007743  0.066623
          2    -0.015554  0.066855
          3    -0.007746  0.066440
          4    -0.011371 -0.055604
          5    -0.019073 -0.056400
          6    -0.011268 -0.055898
          7    -0.019486 -0.056215

In [50]:
job.oardel()

[2018-12-20 18:18:06][DEBUG] [frontend | /home/tocornebize] oardel 1829997
